In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import pandas as pd
import cv2
import random
import os
from os import listdir
from os.path import isfile, join

## Custom SU to COCO conversion

In [ ]:
# Getting all image annotation file path 
annotation_dir = "/content/drive/MyDrive/Official Folder DigitalTwins/Data/Batch-1/Image_annotation_8images_batch1"
img_anno = [join(annotation_dir,f) for f in listdir(annotation_dir) if isfile(join(annotation_dir, f))]

In [ ]:
def new_format(json_path, id):
  ann = {
        "info": "Batch2",
        "categories": [
            {
                "supercategory": "Symbol",
                "id": 0,
                "name": "Symbol"
            },
            {
                "supercategory": "Text",
                "id": 1,
                "name": "Text"
            },
            {
                "supercategory": "Line",
                "id": 2,
                "name": "Line"
            }
        ],
        "image": {
            "image_id": None,
            "file_name": None,
            "height": None,
            "width": None
        },
        "annotations": []
    }
  image_name = json_path.split("/")[-1].split(".")[0]
  new_name = "coco_annots_{}.json".format(image_name)
  with open(json_path) as file:
    an = json.load(file)
  ann['image']['image_id'] = id
  ann['image']['file_name'] = an['metadata']['name']
  ann['image']['height'] = an['metadata']['height']
  ann['image']['width'] = an['metadata']['width']

  
  for an_id, j in enumerate(an['instances']):
    if j["className"] == "piping_lines" or j["className"] == "other_lines":
      pass
    else:
      # print(j['points']['x1'])
      new = {
              "area": ann['image']['height']*ann['image']['width'],
              "bbox": None,
              "category_id": 0,
              "annot_id": an_id,
              "image_id": id,
              "iscrowd": 0,
              "segmentation": None,
              "classId": j['classId'],
              "className": j['className'].lower(),
              "value": " "
          }

      # if len(j['points'])> 4:
      #   print(j['className'])
      #   print(j['type'])
      #   print(j['points'])

      if j['type'] == 'bbox':
        new['bbox'] = [j['points']['x1'], j['points']['x2'], j['points']['y1'], j['points']['y2']]
      # else:
      #   new['bbox'] = j['points']
        

      if j['className'].lower() == "inside text" or j['className'] == "text":
        new["className"] = "text"
        try:
          new['value'] = j['pointLabels']['0']
        except:
          new['value'] = " "
      ann['annotations'].append(new)
    # print(ann)
  new_folder = join(annotationsroot_dir, new_name)
  with open(new_folder, 'w') as outfile:
    json.dump(ann, outfile)
  

In [ ]:
for id, path in enumerate(img_anno):
  print(id, path)
  new_format(path, id)

## SU to COCO using Superannotate API

In [ ]:
import superannotate as sa
sa.export_annotation("Annotation folder", "Output folder", "COCO", "Final folder name", project_type='Vector', task='object_detection')